# **Working with DF (DataFrame)**

**`Udemy Course: Best Hands-on Big Data Practices and Use Cases using PySpark`**

**`Author: Amin Karami (PhD, FHEA)`**

---

**DataFrame (DF)**: Schema (named columns) + declarative language. A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. It is very efficient for strucutred data.

source: https://spark.apache.org/docs/latest/sql-programming-guide.html

source: https://spark.apache.org/docs/latest/api/python/reference/

In [0]:
########## ONLY in Colab ##########
!pip3 install pyspark
########## ONLY in Colab ##########

In [0]:
# Linking with Spark (https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html)
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

spark

# **Part 1: Create DF and Basic Operations**

In [0]:
# Create/Load DF: (Spark automatically scans through the files and infers the schema of the dataset)
# data source: https://www.kaggle.com/thec03u5/fifa-18-demo-player-dataset

df1 = spark.read.format("csv").load("CompleteDataset.csv", inferSchema=True, header=True)

In [0]:
# Show data:
df1.show()

In [0]:
# How many partitions in DF?
df1.rdd.getNumPartitions()

In [0]:
# Increase/Desrease the partitions in DF
df2 = df1.repartition(4)
df2.rdd.getNumPartitions()

In [0]:
# Show DF
df2.show()

In [0]:
# Rename Columns and Amend NULLs:
df2 = df2.withColumnRenamed("_c0", "id") \
    .withColumnRenamed("Ball control", "Ball_control") \
    .withColumnRenamed("Sliding tackle", "Sliding_Tackle")

df2.na.fill({"RAM":10, "RB":1}).show()

In [0]:
# Transformation (SELECT):
df2.select("Name", "Overall").distinct().show()

In [0]:
# Transformation (FILTER):
df2.filter(df2["Overall"] > 90).show()

In [0]:
# Transformation (FILTER):
df2.select("Overall", "Name", "Age").where(df2["Overall"] > 90).show()

In [0]:
# Transformation (FILTER):
df2.where(df2["Overall"]>70).groupBy("Age").count().sort("Age").show()

In [0]:
# Visualize the results:
df2_result = df2.where(df2["Overall"]>70).groupBy("Age").count().sort("Age")
pandas_df = df2_result.toPandas()

pandas_df.plot(x="Age", y="count", kind="bar")

In [0]:
pandas_df.sort_values(by="count", ascending=False).plot(x="Age", y="count", kind="bar")

# **Part 2: Advanced DF Operations: Spark SQL and UDF**

In [0]:
# Spark SQL (Register the DF using a local temporary view):
df2.createOrReplaceTempView("df_football")

In [0]:
# SQL Query:

sql_query = """SELECT Age, count(*) as Count
              FROM df_football
              WHERE Overall > 70
              GROUP BY Age
              ORDER BY Age"""

result = spark.sql(sql_query)
result.show()

In [0]:
# UDF (User Defined Functions):
def uppercase_converter(record):
  if len(record) > 10:
    return record.upper()
  else:
    return record.lower()

# register the DF
df2.createOrReplaceTempView("UDF_football")

# register the function
spark.udf.register("UPPER", uppercase_converter)

# use the UDF in SQL
sql_query = "SELECT Age, UPPER(Name) as Name, UPPER(Club) as Club FROM UDF_football"
result = spark.sql(sql_query)
result.show()